In [ ]:

%%capture
import os
from pathlib import Path

import pandas as pd
from dj_notebook import activate

env_file = os.environ["META_ENV"]
reports_folder = Path(os.environ["META_REPORTS_FOLDER"])
analysis_folder = Path(os.environ["META_ANALYSIS_FOLDER"])
pharmacy_folder = Path(os.environ["META_PHARMACY_FOLDER"])
plus = activate(dotenv_file=env_file)

In [ ]:
from edc_pharmacy.analytics import get_next_scheduled_visit_for_subjects_df
from edc_pharmacy.analytics import in_stock_for_subjects_df

In [ ]:

df_next_appts = get_next_scheduled_visit_for_subjects_df()

In [ ]:
df_next_appts

In [ ]:
df_stock = in_stock_for_subjects_df(container_name='bottle128', product_name="Metformin").fillna(pd.NA).groupby('subject_identifier')[
    'code'].agg(','.join).reset_index()

In [ ]:
df = df_next_appts.merge(df_stock[["subject_identifier", "code"]], on="subject_identifier", how="left")

In [ ]:
df[["subject_identifier", "code", "site_id"]]

In [ ]:
from edc_pharmacy.models import Stock
from edc_visit_schedule.models import SubjectScheduleHistory
from django_pandas.io import read_frame
from edc_pharmacy.utils import get_imp_schedule_names
visit_schedule_names, schedule_names = get_imp_schedule_names()

df_schedule = read_frame(
    SubjectScheduleHistory.objects.values(
        "subject_identifier",
        "visit_schedule_name",
        "schedule_name",
        "offschedule_datetime",
    ).filter(
        visit_schedule_name__in=visit_schedule_names,
        schedule_name__in=schedule_names,
        offschedule_datetime__isnull=True,
    )
)

df_stock_on_site = read_frame(
    Stock.objects.values(
        "code", "allocation__registered_subject__subject_identifier"
    ).filter(confirmed_at_site=True, dispensed=False),
    verbose=False,
).rename(
    columns={"allocation__registered_subject__subject_identifier": "subject_identifier"}
)



In [ ]:
df = (
    pd.merge(
        df_schedule[["subject_identifier"]],
        df_stock_on_site,
        on="subject_identifier",
        how="left",
    )
    .fillna(pd.NA)
    # .query("code.isna()")[["subject_identifier", "code"]]
)


In [ ]:
df

In [ ]:
df = df.fillna("")
df = (
    df
    # .query("code.isna()")[["subject_identifier", "code"]]
    .groupby("subject_identifier")["code"]
    .agg(",".join)
    .reset_index()
    .sort_values(by=["subject_identifier"])
    .reset_index(drop=True)
)

In [ ]:
df

In [ ]:
df.query("code.str.strip()==''")